In [1]:
import openslide
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import ast
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa

In [16]:
df = pd.read_csv('../input/mayo-clinic-strip-ai/train.csv')

In [17]:
df['label'] = df['label'].apply(lambda x: 1 if x == 'CE' else 0)

In [18]:
df = df.set_index('image_id')

In [19]:
samples = pd.read_csv('../input/mayo-clinic-strip-ai/test.csv')

In [20]:
final_data= pd.read_csv('../input/final-preprocessing/navigation_v3.csv')
final_data = final_data.set_index('Unnamed: 0')
final_data = final_data.astype(object)

In [21]:
def get_patch(path, tl_pixel, patch_shape):
    img = openslide.open_slide(path)
    img = np.array(img.read_region(tl_pixel, 0, patch_shape).convert("RGB"))
    return cv2.resize(img/255.0, (1024, 1024))

In [22]:
# def get_patch2(os_obj, tl_pixel, patch_shape):
#     return np.array(os_obj.read_region(tl_pixel, 0, patch_shape).convert("RGB"))

# final_data = pd.DataFrame()
# ids = []
# for n in range(df.shape[0]):
#     img_path = "../input/mayo-clinic-strip-ai/train/"+df.iloc[n]['image_id']+".tif"
#     label = df.iloc[n]['label']
#     i_id  = df.iloc[n]['image_id']
#     ids.append(i_id)
#     img  = openslide.open_slide(img_path)
#     series = pd.Series()
#     print(n)
#     for i in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
#         for j in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
#             im = get_patch2(img , (i,j), (512,512) )
#             if im.std()>15.0:
#                 s = pd.Series(data = [(i,j)])
#                 series = series.append(s,ignore_index=True)
#                 series = series.reset_index()
#                 series = series.drop("index",axis=1)
#     final_data = pd.concat([final_data , series],axis=1)
#     final_data.columns = ids


In [23]:
global train_test_list
train_test_list = np.random.choice(np.arange(754),754)

In [117]:
def data_gen(df = df , data = final_data):
    for i in train_test_list[:650]:
        n_images = 4 - final_data.iloc[i].isnull().sum()
        if n_images <2:
            continue
        label = df.loc[final_data.index[i]]['label']
        if label == 0 :
            images_list = np.arange(n_images)
            for x in range(3):
                final_image_list = np.random.choice(images_list, size=(2), replace=False)
                img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(1024,1024))
#                 img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
#                 img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
#                 img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
#                 img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
                yield (img1,label)
        else:
            images_list = np.arange(n_images)
            for x in range(1):
                final_image_list = np.random.choice(images_list, size=(2), replace=False)
                img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(1024,1024))
#                 img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
#                 img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
#                 img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
#                 img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
                yield (img1,label)
       
    
    
def test_gen(df = df , data = final_data):
    for i in train_test_list[650:]:
        n_images = 4 - final_data.iloc[i].isnull().sum()
        if n_images <2:
            continue
        label = df.loc[final_data.index[i]]['label']
        if label == 0 :
            images_list = np.arange(n_images)
            final_image_list = np.random.choice(images_list, size=(2), replace=False)
            img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(1024,1024))
#             img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
#             img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
#             img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
#             img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
            yield (img1,label)
        else:
            images_list = np.arange(n_images)
            final_image_list = np.random.choice(images_list, size=(2), replace=False)
            img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(1024,1024))
#             img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
#             img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
#             img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
#             img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
            yield (img1,label)

In [118]:
dataset = tf.data.Dataset.from_generator(
     data_gen,
     (tf.float32, tf.float32),
    (tf.TensorShape([1024,1024,3]), tf.TensorShape([]))
)
testset = tf.data.Dataset.from_generator(
     test_gen,
     (tf.float32, tf.float32),
    (tf.TensorShape([1024,1024,3]), tf.TensorShape([]))
)

In [119]:

dataset = dataset.batch(1)
dataset = dataset.shuffle(buffer_size = 5, seed=3)
dataset=dataset.prefetch(1)
testset = testset.batch(1)
testset=testset.prefetch(1)

In [101]:
efficient = tf.keras.applications.EfficientNetB5(
    include_top=False,
    weights='imagenet',
    input_shape=(1024,1024,3),
)
def augmentation(A):
    A = tf.keras.layers.RandomFlip()(A)
    A =tf.keras.layers.RandomRotation(
        (-0.2, 0.2),
        fill_mode='reflect',
        interpolation='bilinear',
        seed=None,
        fill_value=0.0,
    )(A)
    A = tf.keras.layers.RandomZoom(
        (-0.1,0.1),
        (-0.1,0.1),
        fill_mode='reflect',
        interpolation='bilinear',
        seed=None,
        fill_value=0.0,
    )(A)
    return A

115277824/115263384 [==============================] - 1s 0us/step


In [102]:
inputA = keras.layers.Input(shape=(1024,1024,3))
# inputB = keras.layers.Input(shape=(512,512,3),name='input_72')
# inputC = keras.layers.Input(shape=(512,512,3),name='input_73')
# inputD = keras.layers.Input(shape=(512,512,3),name='input_74')
# inputE = keras.layers.Input(shape=(512,512,3),name='input_75')

X = augmentation(inputA)
X = efficient(X)
# X = keras.models.Model(inputs=inputA, outputs=X)
# B = augmentation(inputB)
# B = efficient(B)
# B = keras.models.Model(inputs=inputB, outputs=B)
# C = augmentation(inputC)
# C = efficient(C)
# C = keras.models.Model(inputs=inputC, outputs=C)
# D = efficient(inputD)
# D = keras.models.Model(inputs=inputD, outputs=D)
# E = efficient(inputE)
# E = keras.models.Model(inputs=inputE, outputs=E)

# X = keras.layers.Concatenate(axis=-1)([A.output, B.output, C.output, D.output, E.output])
# X = keras.layers.Conv2D(1024, 4, strides = 4)(X) 
X = keras.layers.Conv2D(512, 4, strides = 4)(X)
X = keras.layers.Flatten()(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Dense(64 ,activation = 'relu')(X)
X = keras.layers.Dense(64 ,activation = 'relu')(X)
X = keras.layers.Dense(32 ,activation = 'relu')(X)
X = keras.layers.Dense(16 ,activation = 'relu')(X)
X = keras.layers.Dense(8 ,activation = 'relu')(X)
X = keras.layers.Dense(1 ,activation = 'sigmoid')(X)
                                               
model = keras.models.Model(inputs=inputA, outputs=X)

In [103]:
model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 1024, 1024, 3)]   0         
_________________________________________________________________
random_flip_11 (RandomFlip)  (None, 1024, 1024, 3)     0         
_________________________________________________________________
random_rotation_11 (RandomRo (None, 1024, 1024, 3)     0         
_________________________________________________________________
random_zoom_11 (RandomZoom)  (None, 1024, 1024, 3)     0         
_________________________________________________________________
efficientnetb5 (Functional)  (None, 32, 32, 2048)      28513527  
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 512)         16777728  
_________________________________________________________________
flatten_9 (Flatten)          (None, 32768)             0  

In [120]:
opt = tfa.optimizers.AdamW(
    weight_decay=1e-4,
    learning_rate = 0.001,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'AdamW',
)
loss = tf.keras.losses.BinaryCrossentropy(
    label_smoothing=0.05,
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_crossentropy'
)
model.compile(optimizer=opt,
              loss=loss,
                metrics=[tf.keras.metrics.AUC()])

In [121]:
!mkdir model

mkdir: cannot create directory ‘model’: File exists


In [122]:
model_checkpoint_callback_LASSO = tf.keras.callbacks.ModelCheckpoint(
    filepath = './model/model',
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
)

In [ ]:
H = model.fit(dataset, validation_data=testset, epochs=5, callbacks=[model_checkpoint_callback_LASSO])

Epoch 1/5
1057/1057 [==============================] - 812s 745ms/step - loss: 0.6885 - auc_10: 0.4871 - val_loss: 1.2445 - val_auc_10: 0.4619
Epoch 2/5
 968/1057 [==========================>...] - ETA: 1:04 - loss: 0.6858 - auc_10: 0.4591

In [ ]:
!mkdir final_model

In [ ]:
!mkdir best_model

In [ ]:
model.save('./final_model/model')

In [ ]:
model.load_weights('./model3/model').expect_partial()
opt = tfa.optimizers.AdamW(
    weight_decay=1e-4,
    learning_rate = 0.00005,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
)
loss = tf.keras.losses.BinaryCrossentropy(
    label_smoothing=0.05,
    reduction=tf.keras.losses.Reduction.AUTO,
)
model.compile(optimizer=opt,
              loss=loss,
                metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), tf.keras.metrics.AUC()])


In [ ]:
model.save('./best_model/model')

In [ ]:
model.evaluate(testset)

In [ ]:
# def get_patch1(img, tl_pixel, patch_shape):
#     img = np.array(img.read_region(tl_pixel, 0, patch_shape).convert("RGB"))
#     return img

# final_samples= pd.DataFrame()
# ids = []
# for n in tqdm(range(samples.shape[0])):
#     img_path = "../input/mayo-clinic-strip-ai/test/"+samples.iloc[n]['image_id']+".tif"
#    #label = samples.iloc[n]['label']
#     i_id = samples.iloc[n]['image_id']
#     ids.append(i_id)
#     img  = openslide.open_slide(img_path)
#     series = pd.Series()
#     n = 0
#     for i in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
#         for j in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
#             if n ==20:
#                 break           
#             im = get_patch1(img , (i,j), (512,512) )
#             if im.std()>10.0:
#                 n=n+1
#                 s = pd.Series(data = [(i,j)])
#                 series = series.append(s,ignore_index=True)
#                 series = series.reset_index()
#                 series = series.drop("index",axis=1)
#     final_samples = pd.concat([final_samples , series],axis=1)
#     final_samples.columns = ids
# final_samples = final_samples.T


In [ ]:
# final_samples

In [ ]:
# samples

In [ ]:
# def sample_gen(df = samples , data = final_samples):
#     for i in range(samples.shape[0]):
#         img1 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][0]),(512,512))
#         img2 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][1]),(512,512))
#         img3 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][2]),(512,512))
#         img4 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][3]),(512,512))
#         img5 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][4]),(512,512))
#         yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5})
# sampleset = tf.data.Dataset.from_generator(
#      sample_gen,
#      ({'input_71':tf.float32,'input_72':tf.float32,'input_73':tf.float32,'input_74':tf.float32,'input_75':tf.float32}),
#     ({'input_71':tf.TensorShape([512,512,3]),'input_72':tf.TensorShape([512,512,3]),'input_73':tf.TensorShape([512,512,3]),'input_74':tf.TensorShape([512,512,3]),'input_75':tf.TensorShape([512,512,3])}))


In [ ]:
# weight_file = "./model"
# model.load_weights(weight_file).expect_partial()
# print("Weights loaded successfully")

In [ ]:
# weight_file = "./model"
# model.load_weights(weight_file).expect_partial()
# print("Weights loaded successfully")